# Day 21

Use NFL play-by-play data from 1999 to create a dataset at the game-team level where I can see the score and the number of different types of scoring plays each team had that made up their final score. Ultimately I want to compare the percentage of score by offense/defense to see how teams perform and if there are any outliers at the weekly, team, or season level.

Current challenges:
- Safeties are hard to parse as there are different conditions that can lead to a safety.

Solutions:
- Get the score from another table, join on game_id and team, and just use the non-defense and special teams touchdowns
- Utilize other variables from the play-by-play data to help with data cleaning

In [7]:
import pandas as pd
import sqlite3
import nfl_data_py as nfl

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

## Scrape Play-by-Play Data

In [8]:
cols = [
    'game_id',
    'season',
    'week',
    'home_team',
    'away_team',
    'posteam',
    'defteam',
    'touchdown',
    'pass_touchdown',
    'rush_touchdown',
    'return_touchdown',
    'extra_point_attempt',
    'extra_point_result',
    'two_point_attempt',
    'field_goal_attempt',
    'field_goal_result',
    'two_point_conv_result',
    'safety',
    'success',
    'td_team',
    'posteam_score',
    'defteam_score',
    'posteam_score_post',
    'defteam_score_post',
    'desc']

pbp = nfl.import_pbp_data(range(1999, 2023), cols, downcast=True, cache=False, alt_path=None)
pbp.to_sql('weekly', conn, index=False, if_exists='replace')

1999 done.
2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


/Volumes/develop/projects/fantasy_football_analysis/env/lib/python3.10/site-packages/nfl_data_py/__init__.py:137: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  plays.loc[:, cols] = plays.loc[:, cols].astype(numpy.float32)


## Get Score Summary Table

In [9]:
query = """
WITH offense AS (
    SELECT
        game_id,
        season,
        week,
        home_team,
        away_team,
        posteam,
        SUM(touchdown) AS tot_tds,
        SUM(pass_touchdown) AS tot_pass_tds,
        SUM(rush_touchdown) AS tot_rush_tds,
        SUM(return_touchdown) AS tot_ret_tds
    FROM pbp
    WHERE posteam IS NOT NULL 
        AND posteam <> ""
        AND posteam = td_team
    GROUP BY game_id, posteam), 
extra_pts AS (
    SELECT
        game_id,
        posteam,
        COUNT(*) AS tot_extra_pts
    FROM pbp
    WHERE extra_point_attempt = 1 AND extra_point_result = 'good'
    GROUP BY game_id, posteam),
field_goals AS (
    SELECT 
        game_id,
        posteam,
        COUNT(*) AS tot_field_goals
    FROM pbp
    WHERE field_goal_attempt = 1 AND field_goal_result = 'made'
    GROUP BY game_id, posteam), 
two_pt_convs AS (
    SELECT 
        game_id,
        posteam,
        COUNT(*) AS tot_2pt_conv
    FROM pbp
    WHERE two_point_attempt = 1 AND two_point_conv_result = 'success'
    GROUP BY game_id, posteam),
-- Counts defensive TDs and punt/kickoff return TDs
defense AS (
    SELECT
        game_id,
        td_team AS team,
        COUNT(*) AS tot_def_tds
    FROM pbp
    WHERE td_team = defteam
    GROUP BY game_id, td_team),
safeties AS (
    SELECT
        game_id,
        defteam AS team,
        COUNT(*) AS tot_safeties
    FROM pbp
    WHERE safety = 1
    GROUP BY game_id, defteam),
joined AS (
    SELECT 
        offense.*,
        CASE WHEN tot_extra_pts IS NULL THEN 0
        ELSE tot_extra_pts
        END AS tot_extra_pts,
        CASE WHEN tot_field_goals IS NULL THEN 0
        ELSE tot_field_goals
        END AS tot_field_goals,
        CASE WHEN tot_2pt_conv IS NULL THEN 0
        ELSE tot_2pt_conv
        END AS tot_2pt_conv,
        CASE WHEN tot_def_tds IS NULL THEN 0
        ELSE tot_def_tds
        END AS tot_def_tds,
        CASE WHEN tot_safeties IS NULL THEN 0
        ELSE tot_safeties
        END AS tot_safeties
    FROM offense
    LEFT JOIN extra_pts
        ON extra_pts.game_id = offense.game_id
            AND extra_pts.posteam = offense.posteam
    LEFT JOIN field_goals
        ON field_goals.game_id = offense.game_id
            AND field_goals.posteam = offense.posteam
    LEFT JOIN two_pt_convs
        ON two_pt_convs.game_id = offense.game_id
            AND two_pt_convs.posteam = offense.posteam
    LEFT JOIN defense
        ON defense.game_id = offense.game_id
            AND defense.team = offense.posteam
    LEFT JOIN safeties
        ON safeties.game_id = offense.game_id
            AND safeties.team = offense.posteam
)
SELECT *,
    (tot_pass_tds * 6
    + tot_rush_tds * 6
    + tot_ret_tds * 6
    + tot_extra_pts * 1
    + tot_field_goals * 3
    + tot_2pt_conv * 2
    + tot_def_tds * 6
    + tot_safeties * 2) AS score
FROM joined
"""

df_pbp = pd.read_sql(query, conn)
df_pbp.head(10)

,game_id,season,week,home_team,away_team,posteam,tot_tds,tot_pass_tds,tot_rush_tds,tot_ret_tds,tot_extra_pts,tot_field_goals,tot_2pt_conv,tot_def_tds,tot_safeties,score
0,1999_01_ARI_PHI,1999,1,PHI,ARI,ARI,2.0,1.0,1.0,0.0,1,4,0,0,0,25.0
1,1999_01_ARI_PHI,1999,1,PHI,ARI,PHI,3.0,2.0,1.0,0.0,3,1,0,0,0,24.0
2,1999_01_BUF_IND,1999,1,IND,BUF,BUF,1.0,1.0,0.0,0.0,0,2,1,0,0,14.0
3,1999_01_BUF_IND,1999,1,IND,BUF,IND,3.0,2.0,1.0,0.0,4,1,0,1,0,31.0
4,1999_01_CAR_NO,1999,1,NO,CAR,CAR,1.0,1.0,0.0,0.0,1,1,0,0,0,10.0
5,1999_01_CAR_NO,1999,1,NO,CAR,NO,1.0,1.0,0.0,0.0,1,2,0,1,0,19.0
6,1999_01_CIN_TEN,1999,1,TEN,CIN,CIN,4.0,2.0,2.0,0.0,1,2,2,0,0,35.0
7,1999_01_CIN_TEN,1999,1,TEN,CIN,TEN,4.0,3.0,1.0,0.0,4,2,0,0,1,36.0
8,1999_01_DAL_WAS,1999,1,WAS,DAL,DAL,6.0,5.0,1.0,0.0,5,0,0,0,0,41.0
9,1999_01_DAL_WAS,1999,1,WAS,DAL,WAS,4.0,2.0,2.0,0.0,3,2,1,0,0,35.0
